In [1]:
import pandas as pd
import numpy as np
import cvxpy as cvx
import helper
import project_helper
import os
import pandas_datareader.data as pdr
import datetime as dt
import requests_cache
from alpha_vantage.timeseries import TimeSeries
# quandl key
f = open("/Users/manwaiwong/key/quandl.txt","r")

if f.mode == 'r':
    c = f.read() 

In [2]:
# quandl 
tickers = ["HKEX/00388","HKEX/01810","HKEX/02342", "HKEX/00552"]
expire_after = dt.timedelta(days=1)
session = requests_cache.CachedSession(cache_name='cache', backend='sqlite', expire_after=expire_after)
start = dt.date.today() - dt.timedelta(1095)
end = dt.date.today() 
qkey = c


close = pd.DataFrame() # dataframe to store close price of each ticker
volumes = pd.DataFrame()
dividends = pd.DataFrame()
attempt = 0 # initializing passthrough variable
drop = [] # initializing list to store tickers whose close price was successfully extracted
## loop tickers get data
while len(tickers) != 0 and attempt <= 5:
    tickers = [j for j in tickers if j not in drop] # removing stocks whose data has been extracted from the ticker list
    for i in range(len(tickers)):
        try:
            temp = pdr.DataReader(tickers[i], 'quandl', start, end, api_key=qkey, session=session)
            #print(start)
            #print(end)
            #print(temp)
            close[tickers[i]] = temp["PreviousClose"]
            volumes[tickers[i]] = temp["ShareVolume000"]
            dividends[tickers[i]] = temp["PEx"].fillna(0)
            #temp.dropna(inplace = True)
            drop.append(tickers[i])       
        except:
            print(tickers[i]," :failed to fetch data...retrying")
            continue
    attempt+=1




In [3]:
close

,HKEX/00388,HKEX/01810,HKEX/02342,HKEX/00552
Date,,,,
2020-02-14,268.8,13.38,3.11,5.84
2020-02-13,270.4,12.92,3.11,5.93
2020-02-12,270.2,12.78,3.12,5.80
2020-02-11,268.4,12.96,3.16,5.82
2020-02-10,270.6,12.84,3.19,5.71
...,...,...,...,...
2017-02-21,200.8,NaN,1.50,5.02
2017-02-20,200.2,NaN,1.45,5.04
2017-02-17,202.4,NaN,1.49,5.19
